<a href="https://colab.research.google.com/github/Baldros/BirdCLEFF-2024/blob/main/Extract_BirdCLEF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação para comunicação com o Drive:
from google.colab import drive

# Acessando o Drive:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Criando um diretório:
!mkdir -p /root/.kaggle

# Copiando o Kaggle Json:
!cp '/content/drive/MyDrive/kaggle.json' '/root/.kaggle/'

# Verificando se o arquivo foi copiado corretamente:
!ls /root/.kaggle/

kaggle.json


In [ ]:
!kaggle competitions download -c birdclef-2024

100% 21.8G/21.8G [03:27<00:00, 96.1MB/s]
100% 21.8G/21.8G [03:27<00:00, 113MB/s] 


In [ ]:
# Importa a biblioteca para manipulação de arquivos
from tqdm import tqdm
import zipfile

In [ ]:
# Define o caminho para o arquivo ZIP baixado
zip_file_path = "/content/birdclef-2024.zip"

# Define o diretório de destino para a extração
extract_dir = "/content/birdclef-2024"

# Extraindo arquivo:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Obtenha uma lista dos nomes dos arquivos no arquivo zip
    file_list = zip_ref.namelist()
    # Use tqdm para criar uma barra de progressão
    for file in tqdm(file_list, desc="Extracting"):
        zip_ref.extract(file, extract_dir)

Extracting: 100%|██████████| 32907/32907 [04:45<00:00, 115.22it/s]


In [ ]:
import os
import random
import librosa
import pandas as pd

In [ ]:
# Espécies:
especies = ['asbfly', 'zitcis1', 'eurcoo', 'bkrfla1']
base_path = '/content/birdclef-2024/train_audio'

In [ ]:
def get_audio_files(base_path, especies, num_files=20):
    audio_files = {}

    for especie in especies:
        # Caminho da pasta da espécie
        especie_path = os.path.join(base_path, especie)

        # Listar todos os arquivos na pasta da espécie
        all_files = [os.path.join(especie_path, file) for file in os.listdir(especie_path) if file.endswith('.ogg')]

        # Selecionar aleatoriamente 20 arquivos (ou o número total se houver menos de 20)
        selected_files = random.sample(all_files, min(len(all_files), num_files))

        # Adicionar à lista de arquivos
        audio_files[especie] = selected_files

    return audio_files

In [ ]:
# Obter os caminhos dos arquivos
audio_files = get_audio_files(base_path, especies)

In [ ]:
audio_files.keys()

dict_keys(['asbfly', 'zitcis1', 'eurcoo', 'bkrfla1'])

In [ ]:
# Remover pastas existentes, se houver
!rm -rf sinais
!rm -rf durações

# Criar novas pastas
!mkdir sinais
!mkdir durações

In [ ]:
for key in tqdm(audio_files.keys()):
  for i,audio in enumerate(audio_files[key]):
    y, sr = librosa.load(audio)
    duracao = librosa.get_duration(y=y, sr=sr)

    df_y = pd.DataFrame(y)
    df_duracao = pd.DataFrame([duracao])

    df_y.to_csv(f'/content/sinais/sinal_{key}_{i+1}.csv',index=False)
    df_duracao.to_csv(f'/content/durações/duracao_{key}_{i+1}.csv',index=False)

100%|██████████| 4/4 [01:56<00:00, 29.09s/it]


In [ ]:
def zip_directory(directory_path, zip_file_path):
    # Cria o arquivo ZIP
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Percorre todos os arquivos do diretório
        for root, dirs, files in os.walk(directory_path):
            for file in tqdm(files, desc="Zipping"):
                # Obtém o caminho completo do arquivo
                file_path = os.path.join(root, file)
                # Adiciona o arquivo ao ZIP
                zipf.write(file_path, os.path.relpath(file_path, directory_path))

In [ ]:
# Compacta o diretório
zip_directory('/content/sinais', '/content/sinais.zip')

Zipping: 100%|██████████| 80/80 [01:18<00:00,  1.02it/s]


In [ ]:
zip_directory('/content/durações', '/content/duracoes.zip')

Zipping: 100%|██████████| 80/80 [00:00<00:00, 4395.91it/s]


In [ ]:
y, sr = librosa.load('/content/birdclef-2024/train_audio/asbfly/XC134896.ogg')
duration = librosa.get_duration(y=y, sr=sr)

27.350249433106576

In [ ]:
pd.read_csv('/content/sinais/sinal_asbfly_1.csv')

,0
0,-0.000011
1,-0.000011
2,0.000016
3,0.000003
4,0.000008
...,...
693500,-0.000008
693501,0.000013
693502,0.000022
693503,-0.000004


from matplotlib import pyplot as plt
_df_0['0'].plot(kind='hist', bins=20, title='0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['0'].plot(kind='line', figsize=(8, 4), title='0')
plt.gca().spines[['top', 'right']].set_visible(False)